In [ ]:
!pip install visualkeras==0.1.4 keras_tuner==1.4.7 keras_cv==0.9.0 pydot==3.0.4 graphviz==0.20.3 git+https://github.com/raghakot/keras-vis.git

In [ ]:
# Kagglehub
import kagglehub

# Math Modules
import numpy as np
import pandas as pd

# Graph Plotting Modules
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import ImageOps

# Other Python Modules
import os
import shutil
import warnings
import ssl
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Sci Kit Learn
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Tensorflow/Keras
import visualkeras
import tensorflow as tf
import keras_tuner as kt
from keras_cv.layers import Augmenter, RandomFlip, RandomRotation
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import L2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Conv2D, Flatten, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling

# Seaborn Style Settings
sns.set_context("talk")
sns.set_style(
    "whitegrid", {'axes.facecolor': '#F0F0D7', 'figure.facecolor': '#727D73'})

# Matplotlib Style Settings
mpl.rcParams['text.color'] = 'w'
mpl.rcParams['xtick.color'] = 'w'
mpl.rcParams['ytick.color'] = 'w'
mpl.rcParams['axes.labelcolor'] = 'w'

# Hyperparameters
AUTO = tf.data.AUTOTUNE

# Change Data Type for Efficiency
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
# See if GPU is available
print("Num. GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# # Paths if executed in Google Colab
# path_data = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")
# path_data += "/dataset"
# path_tuner = '/content/drive/MyDrive/Self Projects/Eye Disease Classification DL/Hyperparameter Tuning'
# path_model = '/content/drive/MyDrive/Self Projects/Eye Disease Classification DL/Models'

# Paths if executed in Kaggle
path_data = "/kaggle/input/eye-diseases-classification/dataset"

src_tuner = '/kaggle/input/eye-disease-classification-hyperparameter-tuning/tensorflow2/default/1'
src_models = '/kaggle/input/eye-disease-classification-model-and-training-info/tensorflow2/default/1'

path_tuner = '/kaggle/working/eye-disease-classification-hyperparameter-tuning/tensorflow2/default/1'
path_model = '/kaggle/working/eye-disease-classification-model-and-training-info/tensorflow2/default/1'

if not os.path.isdir(path_tuner):
    shutil.copytree(src_tuner, path_tuner)
if not os.path.isdir(path_model):
    shutil.copytree(src_models, path_model)

# # Paths if executed in Local PC
# path_data = "./dataset"
# path_tuner = './Hyperparameter Tuning/'
# path_model = './Models/'

In [ ]:
# Importing Dataset & Setting Them In Train, Validation and Test Variables
train_ds = image_dataset_from_directory(
    directory = path_data,
    image_size = (128, 128),
    validation_split = 0.3,
    subset = "training",
    seed = 1,
    shuffle = True
)
val_ds = image_dataset_from_directory(
    directory = path_data,
    image_size = (128, 128),
    validation_split = 0.3,
    subset = "validation",
    seed = 1,
    shuffle = True
)
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)
val_ds = val_ds.skip((2*val_batches) // 3)

# Get Class Names
class_names = train_ds.class_names
class_names = [i.replace("_", " ").capitalize() for i in class_names]
print('\nClass Names: ' , class_names)

# Data Shape
print("\nTraining data shape:", train_ds.element_spec)
print("Validation data shape:", val_ds.element_spec)
print("Test data shape:", test_ds.element_spec)

In [ ]:
# Figure Setup
fig, axs = plt.subplots(3, 3, figsize=(10, 10), dpi=80)
fig.suptitle('Sample Images', fontsize=30)

# Plot 9 Different images
for images, labels in train_ds.take(1):
    for i in range(9):
        axs = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[labels[i]] + " (" + labels[i].numpy().astype("str") + ")")
        plt.axis("off")

# Show Images
plt.tight_layout()
plt.show()

In [ ]:
# Count of each Class in train_ds
dataset_unbatched = tuple(train_ds.unbatch())
labels = []
for (image,label) in dataset_unbatched:
    labels.append(class_names[label.numpy()])
labels = pd.Series(labels)
count = labels.value_counts()

count

In [ ]:
del dataset_unbatched, labels, count

In [ ]:
# Dataset Augmenter
augmenter = Augmenter(
    layers=[
        RandomFlip(seed=123),
        RandomRotation(factor=(-1.0, 1.0), fill_mode="constant", seed=123)
    ]
)

# Function To Augment Dataset
def augment_data(images, labels):
    inputs = {"images": images, "labels": labels}
    outputs = augmenter(inputs)
    return outputs['images'], outputs['labels']


# Map 'train_ds' To Augment Dataset
train_ds_augment = train_ds.map(augment_data, num_parallel_calls=AUTO)

# Get Sample Images
sample_images, sample_labels = next(iter(train_ds_augment))

# Plot 9 Different Sample Images
# Plot Size
plt.figure(figsize=(10, 10))

# For Loop To Plot Images
for i, (image, label) in enumerate(zip(sample_images[:9], sample_labels[:9])):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image.numpy().squeeze().astype(float) / 255)
    plt.axis("off")

## Combine New Data With Old Data

# Cast the images and labels to float32 in both datasets
train_ds = train_ds.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.float32)))
train_ds_augment = train_ds_augment.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.float32)))

# Now concatenate the datasets
train_ds = train_ds.concatenate(train_ds_augment)

del train_ds_augment, sample_images, sample_labels, image, label

In [ ]:
# Normalize Data
normalization_layer = Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Configure Data Performance
train_ds = train_ds.cache().prefetch(buffer_size=AUTO)
val_ds = val_ds.cache().prefetch(buffer_size=AUTO)
test_ds = test_ds.cache().prefetch(buffer_size=AUTO)

# Configure memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
    except:
        pass


# Hyperparameter Tuning Function
def model_builder(hp):

    # Hyperparameters
    hp_Neuron1 = hp.Int('Neuron1', min_value=32, max_value=512, step=32)
    hp_Neuron2 = hp.Int('Neuron2', min_value=32, max_value=512, step=32)
    hp_Neuron3 = hp.Int('Neuron3', min_value=32, max_value=512, step=32)
    hp_Neuron4 = hp.Int('Neuron4', min_value=32, max_value=512, step=32)
    hp_Neuron5 = hp.Int('Neuron5', min_value=32, max_value=512, step=32)
    hp_kernel_initializer = hp.Choice('kernel_initializer', values=[
                                      'he_uniform', 'glorot_normal'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Model Setup
    model = Sequential(name='Model_1',layers=[
        # Input & First Convolution
        Conv2D(filters=hp_Neuron1, kernel_size=5, input_shape=(128, 128, 3),
               activation='relu', kernel_initializer=hp_kernel_initializer, kernel_regularizer=L2(0.001)),
        BatchNormalization(),
        AveragePooling2D(pool_size=(2, 2)),

        # 2nd Convolution Layer
        Conv2D(hp_Neuron2, kernel_size=4, kernel_initializer=hp_kernel_initializer,
               activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),
        AveragePooling2D(pool_size=(2, 2)),

        # 3rd Convolution Layer
        Conv2D(hp_Neuron3, kernel_size=3, kernel_initializer=hp_kernel_initializer,
               activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),
        AveragePooling2D(pool_size=(2, 2)),
        Dropout(0.35, seed=123),

        # Flatten
        Flatten(),

        # 4th Dense Layer
        Dense(hp_Neuron4, kernel_initializer=hp_kernel_initializer,
              activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),

        # 5th Dense Layer
        Dense(hp_Neuron5, kernel_initializer=hp_kernel_initializer,
              activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),

        # Output
        Dense(4, activation='softmax', name='output_model_1')
    ])

    # Model Compile
    model.compile(optimizer=Nadam(learning_rate=hp_learning_rate),
                  loss=SparseCategoricalCrossentropy(
                      from_logits=True),
                  metrics=['accuracy'])

    return model


# Tuner
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=50,
    directory=path_tuner,
    project_name='Model_1',
    seed=12345
)

# Callbacks
stop_early = EarlyStopping(monitor='val_loss', patience=5)

# Commence Hyperparameter Tuning
tuner.search(train_ds, epochs=3, batch_size=32,
             validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print Results
print(f"""
Neuron 1: {best_hps.get('Neuron1')}
Neuron 2: {best_hps.get('Neuron2')}
Neuron 3: {best_hps.get('Neuron3')}
Neuron 4: {best_hps.get('Neuron4')}
Neuron 5: {best_hps.get('Neuron5')}
Learning Rate: {best_hps.get('learning_rate')}
Kernal Initialiser: {best_hps.get('kernel_initializer')}
""")

In [ ]:
# Callbacks
monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=30)
csv_logger = CSVLogger(path_model + '/Model_1_Training.csv',
                       separator=',', append=False)

# If Model & CSV Exists, Skip Model Fitting, Else, Fit Model
if os.path.exists(path_model + '/Model_1_Training.h5') and os.path.exists(path_model + '/Model_1_Training.csv'):

    # Load Model
    model_1 = load_model(path_model + '/Model_1_Training.h5')

    # Show Model Summary
    model_1.summary()

else:

    # Get Model From Hyperparameter
    model_1 = tuner.hypermodel.build(best_hps)

    # Show Model Summary
    model_1.summary()

    # Fit Model
    history = model_1.fit(train_ds, epochs=1000,
                        validation_data=val_ds,
                        callbacks=[monitor_val_acc, csv_logger])

    # Save Model
    model_1.save(path_model + '/Model_1_Training.h5')

# Load Fitting History
history_1 = pd.read_csv(path_model + '/Model_1_Training.csv',
                      sep=',', engine='python')

In [ ]:
# Model Visualization
visualkeras.layered_view(model_1, legend=True,background_fill='#727D73', font_color='white')

In [ ]:
# Show Model Plot
plot_model(model_1, "model_1.png", show_shapes=True, show_layer_names=True,
            show_dtype=True, rankdir='LR')

In [ ]:
# Accuracy Over Epoch Graph
# Figure Setup
fig, axs = plt.subplots(figsize=(10, 5), dpi=80)

# Plot Accuracy and Val_accuracy Graphs
plt.plot(history_1['accuracy'], label='Train')
plt.plot(history_1['val_accuracy'], label='Validation')

# Labels
plt.title('Accuracy of Model_1', fontsize=25)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right', labelcolor='black')

# Show Graph
plt.show()

In [ ]:
# Checking For Overfitting
# Figure Setup
plt.figure(figsize=(10, 5), dpi=80)

# Plot Accuracy and Val_accuracy loss
plt.plot(history_1['loss'])
plt.plot(history_1['val_loss'])

# Labels
plt.title('Loss of Model_1', fontsize=25)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation', 'Test'],
           loc='upper right', labelcolor='black')

# Show Graph
plt.show()

In [ ]:
# Final Testing Dataset Evaluation
# Evaluate Model
test_loss, test_acc = model_1.evaluate(test_ds, verbose=2)

# Get Prediction Percentage
prediction = model_1.predict(test_ds)
y_classes = prediction.argmax(axis=-1)

# Figure Setup
sns.set_context('talk')
fig, axs = plt.subplots(4, 4, figsize=(
    50, 50), dpi=80, constrained_layout=True)
fig.suptitle('Random Images with Model\'s Prediction Percentage', fontsize=100)

# Get all data from 'test_ds' instead of in batches
x_test = np.concatenate([x for x, y in test_ds], axis=0)
y_test = np.concatenate([y for x, y in test_ds], axis=0)

# Random Number Generator
ran = np.random.randint(0, high=len(x_test), size=(16), dtype=int)

# Counter
cou = 0

# For Loop To Display Random Images With Percentage
for i in range(0, 4, 2):
    for v in range(0, 4):
        # Set Random Images
        axs[i, v].imshow(x_test[ran[cou]].astype(float))
        n = class_names[y_test[ran[cou]]]
        axs[i, v].set_title(n, fontsize=50)
        axs[i, v].axis('off')

        # Change Colour of Bars According To if Prediction was Right or Wrong
        if y_classes[ran[cou]] == y_test[ran[cou]].astype(int):

            # Get Predictions For Above Images
            axs[i+1, v].bar(class_names, prediction[ran[cou]], color='green')
            axs[i+1, v].set_xticklabels(class_names, rotation=90, fontsize=50)
            cou = cou + 1
        else:

            # Get Predictions For Above Images
            axs[i+1, v].bar(class_names, prediction[ran[cou]], color='red')
            axs[i+1, v].set_xticklabels(class_names, rotation=90, fontsize=50)
            cou = cou + 1

# Show Plot
plt.show()

In [ ]:
# Confusion matrix
plt.rcParams["axes.grid"] = False
cm = confusion_matrix(y_test, y_classes)
ConfusionMatrixDisplay(cm, display_labels=class_names).plot(xticks_rotation='vertical')
plt.rcParams["axes.grid"] = True

# Classification report
report_m1 = classification_report(y_test, y_classes, target_names=class_names)
print(report_m1)

In [ ]:
# Hyperparameter Tuning Function
def model_builder_2(hp):

    # Hyperparameters
    hp_Neuron1 = hp.Int('Neuron1', min_value=32, max_value=512, step=32)
    hp_Neuron2 = hp.Int('Neuron2', min_value=32, max_value=512, step=32)
    hp_Neuron3 = hp.Int('Neuron3', min_value=32, max_value=512, step=32)
    hp_Neuron4 = hp.Int('Neuron4', min_value=32, max_value=512, step=32)
    hp_Neuron5 = hp.Int('Neuron5', min_value=32, max_value=512, step=32)
    hp_kernel_initializer = hp.Choice('kernel_initializer', values=[
                                      'he_uniform', 'glorot_normal'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Model Setup
    model = Sequential(name='Model_2', layers=[
        # Input & First Convolution
        Conv2D(filters=hp_Neuron1, kernel_size=5, input_shape=(128, 128, 3),
               activation='relu', kernel_initializer=hp_kernel_initializer, kernel_regularizer=L2(0.001)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        # 2nd Convolution Layer
        Conv2D(hp_Neuron2, kernel_size=4, kernel_initializer=hp_kernel_initializer,
               activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        # 3rd Convolution Layer
        Conv2D(hp_Neuron3, kernel_size=3, kernel_initializer=hp_kernel_initializer,
               activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.35, seed=123),

        # Flatten
        Flatten(),

        # 4th Dense Layer
        Dense(hp_Neuron4, kernel_initializer=hp_kernel_initializer,
              activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),

        # 5th Dense Layer
        Dense(hp_Neuron5, kernel_initializer=hp_kernel_initializer,
              activation='relu', kernel_regularizer=L2(0.001)),
        BatchNormalization(),

        # Output
        Dense(4, activation='softmax', name='output_model_2')
    ])

    # Model Compile
    model.compile(optimizer=Nadam(learning_rate=hp_learning_rate),
                  loss=SparseCategoricalCrossentropy(
                      from_logits=True),
                  metrics=['accuracy'])

    return model


# Tuner
tuner_2 = kt.RandomSearch(
    model_builder_2,
    objective='val_accuracy',
    max_trials=50,
    directory=path_tuner,
    project_name='Model_2',
    seed=12345
)

# Callbacks
stop_early = EarlyStopping(monitor='val_loss', patience=5)

# Commence Hyperparameter Tuning
tuner_2.search(train_ds, epochs=3, batch_size=32,
             validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner_2.get_best_hyperparameters(num_trials=1)[0]

# Print Results
print(f"""
Neuron 1: {best_hps.get('Neuron1')}
Neuron 2: {best_hps.get('Neuron2')}
Neuron 3: {best_hps.get('Neuron3')}
Neuron 4: {best_hps.get('Neuron4')}
Neuron 5: {best_hps.get('Neuron5')}
Learning Rate: {best_hps.get('learning_rate')}
Kernal Initialiser: {best_hps.get('kernel_initializer')}
""")

In [ ]:
# Callbacks
monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=30)
csv_logger = CSVLogger(path_model + '/Model_2_Training.csv',
                       separator=',', append=False)

# If Model & CSV Exists, Skip Model Fitting, Else, Fit Model
if os.path.exists(path_model + '/Model_2_Training.h5') and os.path.exists(path_model + '/Model_2_Training.csv'):

    # Load Model
    model_2 = load_model(path_model + '/Model_2_Training.h5')

    # Show Model Summary
    model_2.summary()

else:

    # Get Model From Hyperparameter
    model_2 = tuner_2.hypermodel.build(best_hps)

    # Show Model Summary
    model_2.summary()

    # Fit Model
    history = model_2.fit(train_ds, epochs=1000,
                        validation_data=val_ds,
                        callbacks=[monitor_val_acc, csv_logger])

    # Save Model
    model_2.save(path_model + '/Model_2_Training.h5')

# Load Fitting History
history_2 = pd.read_csv(path_model + '/Model_2_Training.csv',
                      sep=',', engine='python')

In [ ]:
# Model Visualization
visualkeras.layered_view(model_2, legend=True,background_fill='#727D73', font_color='white')

In [ ]:
# Show Model Plot
plot_model(model_2, "model_2.png", show_shapes=True, show_layer_names=True,
            show_dtype=True, rankdir='LR')

In [ ]:
# Accuracy Over Epoch Graph
# Figure Setup
fig, axs = plt.subplots(figsize=(10, 5), dpi=80)

# Plot Accuracy and Val_accuracy Graphs
plt.plot(history_2['accuracy'], label='Train')
plt.plot(history_2['val_accuracy'], label='Validation')

# Labels
plt.title('Accuracy of Model_2', fontsize=25)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right', labelcolor='black')

# Show Graph
plt.show()

In [ ]:
# Checking For Overfitting
# Figure Setup
plt.figure(figsize=(10, 5), dpi=80)

# Plot Accuracy and Val_accuracy loss
plt.plot(history_2['loss'])
plt.plot(history_2['val_loss'])

# Labels
plt.title('Loss of Model_2', fontsize=25)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation', 'Test'],
           loc='upper right', labelcolor='black')

# Show Graph
plt.show()

In [ ]:
# Final Testing Dataset Evaluation
# Evaluate Model
test_loss_2, test_acc_2 = model_2.evaluate(test_ds, verbose=2)

# Get Prediction Percentage
prediction_2 = model_2.predict(test_ds)
y_classes_2 = prediction_2.argmax(axis=-1)

# Figure Setup
sns.set_context('talk')
fig, axs = plt.subplots(4, 4, figsize=(
    50, 50), dpi=80, constrained_layout=True)
fig.suptitle('Random Images with Model\'s Prediction Percentage', fontsize=100)

# Get all data from 'test_ds' instead of in batches
x_test = np.concatenate([x for x, y in test_ds], axis=0)
y_test = np.concatenate([y for x, y in test_ds], axis=0)

# Random Number Generator
ran = np.random.randint(0, high=len(x_test), size=(16), dtype=int)

# Counter
cou = 0

# For Loop To Display Random Images With Percentage
for i in range(0, 4, 2):
    for v in range(0, 4):
        # Set Random Images
        axs[i, v].imshow(x_test[ran[cou]].astype(float))
        n = class_names[y_test[ran[cou]]]
        axs[i, v].set_title(n, fontsize=50)
        axs[i, v].axis('off')

        # Change Colour of Bars According To if Prediction was Right or Wrong
        if y_classes_2[ran[cou]] == y_test[ran[cou]].astype(int):

            # Get Predictions For Above Images
            axs[i+1, v].bar(class_names, prediction_2[ran[cou]], color='green')
            axs[i+1, v].set_xticklabels(class_names, rotation=90, fontsize=50)
            cou = cou + 1
        else:

            # Get Predictions For Above Images
            axs[i+1, v].bar(class_names, prediction_2[ran[cou]], color='red')
            axs[i+1, v].set_xticklabels(class_names, rotation=90, fontsize=50)
            cou = cou + 1

# Show Plot
plt.show()

In [ ]:
# Confusion matrix
cm_2 = confusion_matrix(y_test, y_classes_2)
plt.rcParams["axes.grid"] = False
ConfusionMatrixDisplay(cm_2, display_labels=class_names).plot(xticks_rotation='vertical')
plt.rcParams["axes.grid"] = True

# Classification report
report_m2 = classification_report(y_test, y_classes_2, target_names=class_names)
print(report_m2)

In [ ]:
# Print Test Accuracies
print("Model 1 Test Accuracy:", test_acc, "\nModel 2 Test Accuracy:" , test_acc_2)

# Print Classification Reports
print("\nModel 1 Report:\n", report_m1 , "\n\nModel 2 Report:\n" ,report_m2)

# Plot Confusion Matrix, Accuracy & Loss Plots of both Models
fig, axs = plt.subplots(2, 2, figsize=(
    30, 20), dpi=80, constrained_layout=True)

# axs[0,0] Confusion Matrix Model 1
ConfusionMatrixDisplay(cm, display_labels=class_names).plot(xticks_rotation='vertical', ax=axs[0, 0])
axs[0, 0].set_title("Confusion Matrix Model 1", fontsize=50)
axs[0, 0].grid(False)
axs[0, 0].set_adjustable('datalim')
axs[0, 0].set_facecolor('#727D73')

# axs[0,1] Confusion Matrix Model 2
ConfusionMatrixDisplay(cm_2, display_labels=class_names).plot(xticks_rotation='vertical', ax=axs[0, 1])
axs[0, 1].set_title("Confusion Matrix Model 2", fontsize=50)
axs[0, 1].grid(False)
axs[0, 1].set_adjustable('datalim')
axs[0, 1].set_facecolor('#727D73')

# axs[1,0] Accuracy Over Epoch
axs[1, 0].plot(history_1['accuracy'], label='Train Model 1')
axs[1, 0].plot(history_2['accuracy'], label='Train Model 2')
axs[1, 0].plot(history_1['val_accuracy'], label='Validation Model 1')
axs[1, 0].plot(history_2['val_accuracy'], label='Validation Model 2')
axs[1, 0].set_title('Accuracy', fontsize=50)
axs[1, 0].set_xlabel('Epoch')
axs[1, 0].set_ylabel('Accuracy')
axs[1, 0].set_ylim([0.5, 1])
axs[1, 0].legend(loc='lower right', labelcolor='black')

# axs[1,1] Loss Over Epoch
axs[1, 1].plot(history_1['loss'] , label='loss Model 1')
axs[1, 1].plot(history_2['loss'] , label='loss Model 2')
axs[1, 1].plot(history_1['val_loss'] , label='Validation loss Model 1')
axs[1, 1].plot(history_2['val_loss'] , label='Validation loss Model 2')
axs[1, 1].set_title('Loss', fontsize=50)
axs[1, 1].set_ylabel('Loss')
axs[1, 1].set_xlabel('Epoch')
axs[1, 1].legend(loc='upper right', labelcolor='black')

# Show Plot
plt.show()

In [ ]:
# Create a Sample Dataset with 100% Accuracy
x_sample = np.array([x_test[16],x_test[33],x_test[5],x_test[3]])
y_sample = np.array([y_test[16],y_test[33],y_test[5],y_test[3]])
model_2.evaluate(x_sample, y_sample)

In [ ]:
del x_test, y_test

In [ ]:
# Get Weights of Convolutional Layers
for i in model_2.layers:
    if 'conv' in i.name:
        filters, biases = i.get_weights()
        print(i.name, filters.shape)


# Plotting Convolutional Layers
fig, axs = plt.subplots(3, 3, figsize=(5, 5), dpi=80)
fig.suptitle('Convolutional Layers', fontsize=30)

# Plot 9 Different images
for i in range(9):
    axs = plt.subplot(3, 3, i + 1)
    axs.imshow(filters[:, :, :, i][:, :, 0].squeeze(), cmap='gray')
    axs.axis("off")

# Show Images
plt.tight_layout()
plt.show()

In [ ]:
# Define a new truncated model to only include the conv layers of interest
truncated_model = Sequential()
for layer in model_2.layers:
    if 'conv' in layer.name:
        truncated_model.add(layer)

# Generate feature output by predicting on the input image (SELECT NORMAL)
feature_output = truncated_model.predict(x_sample[0].reshape(1, 128, 128, 3))

columns = 8
rows = 8
for ftr in feature_output:
    fig=plt.figure(figsize=(5, 5))
    fig.suptitle('CNN Filter of: '+class_names[y_sample[0]])
    for i in range(1, columns*rows +1):
        fig =plt.subplot(rows, columns, i)
        fig.set_xticks([])  #Turn off axis
        fig.set_yticks([])
        plt.imshow(ftr[:, :, i-1], cmap='gray')
    plt.show()

In [ ]:
# Generate feature output by predicting on the input image (SELECT NORMAL)
feature_output = truncated_model.predict(x_sample[1].reshape(1, 128, 128, 3))

columns = 8
rows = 8
for ftr in feature_output:
    fig=plt.figure(figsize=(5, 5))
    fig.suptitle('CNN Filter of: '+class_names[y_sample[1]])
    for i in range(1, columns*rows +1):
        fig =plt.subplot(rows, columns, i)
        fig.set_xticks([])  #Turn off axis
        fig.set_yticks([])
        plt.imshow(ftr[:, :, i-1], cmap='gray')
    plt.show()

In [ ]:
# Generate feature output by predicting on the input image (SELECT GLAUCOMA)
feature_output = truncated_model.predict(x_sample[2].reshape(1, 128, 128, 3))

columns = 8
rows = 8
for ftr in feature_output:
    fig=plt.figure(figsize=(5, 5))
    fig.suptitle('CNN Filter of: '+class_names[y_sample[2]])
    for i in range(1, columns*rows +1):
        fig =plt.subplot(rows, columns, i)
        fig.set_xticks([])  #Turn off axis
        fig.set_yticks([])
        plt.imshow(ftr[:, :, i-1], cmap='gray')
    plt.show()

In [ ]:
# Generate feature output by predicting on the input image (SELECT CATARACT)
feature_output = truncated_model.predict(x_sample[3].reshape(1, 128, 128, 3))

columns = 8
rows = 8
for ftr in feature_output:
    fig=plt.figure(figsize=(5, 5))
    fig.suptitle('CNN Filter of: '+class_names[y_sample[3]])
    for i in range(1, columns*rows +1):
        fig =plt.subplot(rows, columns, i)
        fig.set_xticks([])  #Turn off axis
        fig.set_yticks([])
        plt.imshow(ftr[:, :, i-1], cmap='gray')
    plt.show()

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        inputs = model.inputs,
        outputs = [
                    model.get_layer(last_conv_layer_name).output,
                    model.layers[-1].output,
                ]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img, heatmap, alpha_activation=0.4, alpha_img=1):
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Remove the batch dimension by squeezing the image
    img = img.squeeze()

    # Use jet colormap to colorize heatmap
    jet = mpl.colormaps["jet"]

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1]-20, img.shape[0]-20))
    jet_heatmap = ImageOps.expand(jet_heatmap, border=10, fill='black')
    jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = (jet_heatmap * alpha_activation) + ((img * 255) * alpha_img)

    return superimposed_img/superimposed_img.max()

In [ ]:
# Duplicate & Remove last layer's activation
model_2_viz = tf.keras.models.clone_model(model_2)
model_2_viz.layers[-1].activation = None

# Plot Sample Image from Each Class With Gradually increasing Activation Overlap
fig, axs = plt.subplots(3,4, figsize=(10, 10), dpi=80)
fig.suptitle('Sample Image from Each Class with Activation')

# Plot Original Image, Activation Image & Overlap Image
for i in range(4):
    # Reset Alpha
    alpha_activation = 0
    alpha_img = 1

    # Get Image
    img = x_sample[i].reshape(1, 128, 128, 3)

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img, model_2_viz, 'conv2d_5')

    for v in range(3):

      # Plot Images
      axs[v, i].imshow(save_and_display_gradcam(img, heatmap, alpha_activation, alpha_img))
      axs[v, i].set_title(class_names[y_sample[i]])
      axs[v, i].axis("off")

      # Increase Alpha
      alpha_activation+=0.5
      alpha_img-=0.5

# Show Images
plt.tight_layout()
plt.show()